## Old listing testing

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import re

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
            "accept": "application/json"}

resp = requests.get(url='https://www.oldlistings.com.au/real-estate/VIC/Carlton/3053/rent/', headers=headers)
soup = BeautifulSoup(resp.text)

In [4]:
elems = soup.select('div.property.clearfix')
elem = elems[0]

In [5]:
lat = elem['data-lat']
long = elem['data-lng']
address = elem.find("h2", {'class': "address"}).text
beds = int(elem.find('p', {'class':'property-meta bed'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta bed'}) is not None else 0
baths = int(elem.find('p', {'class':'property-meta bath'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta bath'}) is not None else 0
cars = int(elem.find('p', {'class':'property-meta car'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta car'}) is not None else 0
# cars is nonexisting zometimes

category = elem.find('p', {'class':'property-meta type'}).text.split(':')[1].strip()
last_price = elem.find('section', {'class':'price'}).text.split(':')[1].strip() 
historical_prices = [x.find_all(text=True) for x in elem.find('section', {'class':'historical-price'}).find_all('li')]

lat, long, address, beds, baths, cars, category, last_price, historical_prices

('-37.80634001',
 '144.96191001',
 '4605/160 VICTORIA STREET, CARLTON',
 2,
 1,
 0,
 'Unit/apmt',
 'September 2022 $600',
 [['September 2022', '$600'],
  ['March 2022', '$479 per week'],
  ['January 2022', '$479 Weekly']])

In [6]:
columns = ['lat', 'long', 'address', 'beds', 'baths', 'cars', 'type', 'latest_price', 'historical_prices']

In [7]:
for elem in elems:
    lat = elem['data-lat']
    long = elem['data-lng']
    address = elem.find("h2", {'class': "address"}).text
    beds = int(elem.find('p', {'class':'property-meta bed'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta bed'}) is not None else 0
    baths = int(elem.find('p', {'class':'property-meta bath'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta bath'}) is not None else 0
    cars = int(elem.find('p', {'class':'property-meta car'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta car'}) is not None else 0
    # cars is nonexisting zometimes

    category = elem.find('p', {'class':'property-meta type'}).text.split(':')[1].strip() if elem.find('p', {'class':'property-meta type'}) is not None else 0
    last_price = elem.find('section', {'class':'price'}).text.split(':')[1].strip() 
    historical_prices = [x.find_all(text=True) for x in elem.find('section', {'class':'historical-price'}).find_all('li')]
    print(last_price, historical_prices)
    

September 2022 $600 [['September 2022', '$600'], ['March 2022', '$479 per week'], ['January 2022', '$479 Weekly']]
September 2022 Call Agent [['September 2022', 'Call Agent'], ['November 2007', '$27,000 Plus'], ['November 2007', '$324,000 To Let'], ['August 2007', '$27,000 Plus']]
September 2022 $90,000 pa [['September 2022', '$90,000 pa']]
September 2022 $430 per week [['September 2022', '$430 per week'], ['August 2022', '$430 per week'], ['October 2018', '$410 per week'], ['October 2018', '$410'], ['February 2016', '$390'], ['December 2014', '$380']]
September 2022 $330 pw [['September 2022', '$330 pw']]
September 2022 Under Application [['September 2022', 'Under Application']]
September 2022 $350 per week [['September 2022', '$350 per week'], ['September 2022', '$350']]
September 2022 $300 [['September 2022', '$300'], ['August 2022', '$380 p.w.'], ['July 2022', '$380'], ['July 2022', '$450'], ['April 2022', '$350'], ['December 2021', '$350 Negotiable'], ['November 2021', '$160'], ['

In [8]:
next_url = soup.find_all('li', 'next')[0].a['href']

In [9]:
next_url

'/real-estate/VIC/Carlton/3053/rent/2'

In [5]:
BASE_URL = 'https://www.oldlistings.com.au'
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
            "accept": "application/json"}
COLUMNS = ['lat', 'long', 'postal_code', 'address', 'beds', 'baths', 'cars', 'type', 'price', 'date']

def scrape_suburb(suburb, postal_code):
    return scrape_from_ref('/real-estate/VIC/'+suburb+'/'+str(postal_code)+'/rent/')

def scrape_from_ref(url, max_depth=0):
    # if max_depth == 2:
    #     return pd.DataFrame(columns=COLUMNS)
    print(BASE_URL+url)
    resp = requests.get(url=BASE_URL+url, headers=headers)
    soup = BeautifulSoup(resp.text)
    df = pd.DataFrame(columns=COLUMNS)
    elems = soup.select('div.property.clearfix')
    for elem in elems:
        lat = elem['data-lat']
        long = elem['data-lng']
        postal_code = url.split('/')[4]
        address = elem.find("h2", {'class': "address"}).text
        beds = int(elem.find('p', {'class':'property-meta bed'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta bed'}) is not None else -1
        baths = int(elem.find('p', {'class':'property-meta bath'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta bath'}) is not None else -1
        cars = int(elem.find('p', {'class':'property-meta car'}).text.split(':')[1]) if elem.find('p', {'class':'property-meta car'}) is not None else -1
        # cars is nonexisting zometimes

        category = elem.find('p', {'class':'property-meta type'}).text.split(':')[1].strip() if elem.find('p', {'class':'property-meta type'}) is not None else -1
        last_price = elem.find('section', {'class':'price'}).text.split(':')[1].strip() 
        historical_prices = [x.find_all(text=True) for x in elem.find('section', {'class':'historical-price'}).find_all('li')]
        for hist_price in historical_prices:
            df.loc[len(df)] = lat, long, postal_code, address, beds, baths, cars, category, hist_price[1], hist_price[0]

    next_url = soup.find_all('li', 'next')[0].a['href']
    time.sleep(randint(1, 5))

    if next_url:
        print(next_url)
        df1 = scrape_from_ref(next_url, max_depth+1)
        
    return pd.concat([df, df1], ignore_index=True)

df = scrape_from_ref('/real-estate/VIC/Carlton/3053/rent/')


https://www.oldlistings.com.au/real-estate/VIC/Carlton/3053/rent/
/real-estate/VIC/Carlton/3053/rent/2
https://www.oldlistings.com.au/real-estate/VIC/Carlton/3053/rent/2
/real-estate/VIC/Carlton/3053/rent/3


In [6]:
df

,lat,long,postal_code,address,beds,baths,cars,type,price,date
0,-37.80634001,144.96191001,3053,"4605/160 VICTORIA STREET, CARLTON",2,1,0,Unit/apmt,$600,September 2022
1,-37.80634001,144.96191001,3053,"4605/160 VICTORIA STREET, CARLTON",2,1,0,Unit/apmt,$479 per week,March 2022
2,-37.80634001,144.96191001,3053,"4605/160 VICTORIA STREET, CARLTON",2,1,0,Unit/apmt,$479 Weekly,January 2022
3,-37.80550543,144.96335990,3053,"105 / 488 SWANSTON STREET, CARLTON",2,1,1,0,$400 pw,September 2022
4,-37.80550543,144.96335990,3053,"105 / 488 SWANSTON STREET, CARLTON",2,1,1,0,$400 p.w.,September 2022
...,...,...,...,...,...,...,...,...,...,...
636,-37.805429,144.963372,3053,"761/488 SWANSTON STREET, CARLTON",2,1,0,Rental_residential,$280 p.w.,November 2021
637,-37.805429,144.963372,3053,"761/488 SWANSTON STREET, CARLTON",2,1,0,Rental_residential,$480 Week,November 2020
638,-37.805429,144.963372,3053,"761/488 SWANSTON STREET, CARLTON",2,1,0,Rental_residential,$430,November 2020
639,-37.805429,144.963372,3053,"761/488 SWANSTON STREET, CARLTON",2,1,0,Rental_residential,$300 p.w.,November 2020


## Getting urls to scrape

In [33]:
# sitemap_url = 'https://www.oldlistings.com.au/site-map?state=VIC{page}'
# headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
#             "accept": "application/json"}
# resp = requests.get(url=sitemap_url, headers=headers)
# soup = BeautifulSoup(resp.text)


In [90]:
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
            "accept": "application/json"}
SITEMAP_URL = 'https://www.oldlistings.com.au/site-map?{page}state=VIC'
PATTERN = '\/real-estate\/VIC\/\w*\/\d*\/rent\/'

for i in range(18):
    print(i)
    if i != 0:
        url = SITEMAP_URL.format(page='page='+str(i)+'&')
    else:
        url = SITEMAP_URL.format(page='')
    print(url)
    resp = requests.get(url=url, headers=headers)
    urls = re.findall(PATTERN, resp.text)
    
    with open('../data/raw/to_scrape_oldlistings.txt', 'a+') as f:
        for ref in set(urls):
            f.write(ref)
            f.write('\n')

    time.sleep(randint(5, 15))


0
https://www.oldlistings.com.au/site-map?state=VIC
1
https://www.oldlistings.com.au/site-map?page=1&state=VIC
2
https://www.oldlistings.com.au/site-map?page=2&state=VIC
3
https://www.oldlistings.com.au/site-map?page=3&state=VIC
4
https://www.oldlistings.com.au/site-map?page=4&state=VIC
5
https://www.oldlistings.com.au/site-map?page=5&state=VIC
6
https://www.oldlistings.com.au/site-map?page=6&state=VIC
7
https://www.oldlistings.com.au/site-map?page=7&state=VIC
8
https://www.oldlistings.com.au/site-map?page=8&state=VIC
9
https://www.oldlistings.com.au/site-map?page=9&state=VIC
10
https://www.oldlistings.com.au/site-map?page=10&state=VIC
11
https://www.oldlistings.com.au/site-map?page=11&state=VIC
12
https://www.oldlistings.com.au/site-map?page=12&state=VIC
13
https://www.oldlistings.com.au/site-map?page=13&state=VIC
14
https://www.oldlistings.com.au/site-map?page=14&state=VIC
15
https://www.oldlistings.com.au/site-map?page=15&state=VIC
16
https://www.oldlistings.com.au/site-map?page=16&